In [1]:
from common import Common
from pyspark import *
from pyspark.streaming import *
from pyspark import SparkContext, SparkConf

In [2]:
## Do not run this multiple times
common = Common()
sc = common.get_spark_core()

<SparkContext master=local appName=myapp>
2.4.5


In [3]:
rdd = sc.textFile('/home/ec2-user/data/empty_lines.txt')
print ('line count:', rdd.count())
rdd.take(2)

print ('empty lines: ', rdd.filter(lambda x : len(x) == 0).count())

line count: 31
empty lines:  16


## Accumulators

In [4]:
def line_len(line:str):
    if len(line.strip()) == 0:
        global num_empty_lines
        num_empty_lines += 1
    return len(line)

#### Accumulators, take 5

In [5]:
num_empty_lines = sc.accumulator(0)
words = rdd.map(line_len)
print (words.take(5))
print (num_empty_lines.value)

[58, 0, 56, 0, 0]
3


#### Accumulators, take 20

In [6]:
num_empty_lines = sc.accumulator(0)
words = rdd.map(line_len)
print (words.take(20))
print (num_empty_lines.value)

[58, 0, 56, 0, 0, 15, 0, 106, 0, 77, 0, 58, 0, 167, 0, 0, 387, 0, 151, 0]
11


#### Accumulators, collect

In [7]:
num_empty_lines = sc.accumulator(0)
words = rdd.map(line_len)
_coll = (words.collect())
print (num_empty_lines.value)

16


## Broadcast
### Remove stop words

In [8]:
stopwords = ["a", "an", "the", "of", "at", "is", "am", "are", "this", "that", '', 'at', 'for', 'with']
print (stopwords)

['a', 'an', 'the', 'of', 'at', 'is', 'am', 'are', 'this', 'that', '', 'at', 'for', 'with']


In [9]:
swords = sc.broadcast(stopwords)
print ('broadcast: ', swords.value)

broadcast:  ['a', 'an', 'the', 'of', 'at', 'is', 'am', 'are', 'this', 'that', '', 'at', 'for', 'with']


In [10]:
rdd = sc.textFile('/home/ec2-user/data/empty_lines.txt')
print ('line count:', rdd.count())
rdd.take(2)

def toWords(line:str):
    words = line.split(" ")
    words = [i for i in words if i not in swords.value]
    return words

not_stopwords = rdd.flatMap(toWords)
not_stopwords.take(10)


line count: 31


['Apache',
 'Spark',
 'open-source',
 'distributed',
 'general-purpose',
 'cluster-computing',
 'framework.',
 'Spark',
 'provides',
 'interface']